# Symbolic Partial Derivative Routine

## Authors: Zach Etienne, Tyler Knowles & Siddharth Mahesh

## This module contains a routine for computing partial derivatives of a mathematical expression that is written as several subexpressions.

**Notebook Status:** <font color='green'><b> Validated </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation). Additionally, this notebook has been validated by checking that results are consistent with exact derivative expressions used in the SEOBNRv3_opt approixment of [LALSuite](https://git.ligo.org/lscsoft/lalsuite).

### NRPy+ Source Code for this module: [SEOBNR_Derivative_Routine.py](../edit/SEOBNR/SEOBNR_Derivative_Routine.py)

## Introduction
$$\label{intro}$$

This notebook documents the symbolic partial derivative routine used to generate analytic derivatives of the [SEOBNRv3](https://git.ligo.org/lscsoft/lalsuite) Hamiltonian (documented [here](../Tutorial-SEOBNR_v3_Hamiltonian.ipynb)) and described in [this article](https://arxiv.org/abs/1803.06346).  In general, this notebook takes as input a file of inter-dependent mathematical expressions (in SymPy syntax), a file listing the names of values within those expressions, and a file listing all variables with which to take partial derivatives of each expression.  The output is a text file containing the original expression and those for each partial derivative computation.  The intention is to perform CSE on these expressions to create efficient partial derivative code!

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Initialize core Python/NRPy+ modules
1. [Step 2:](#read_expressions) Read in Hamiltonian expressions from `Sympy_Hreal_on_Bottom.txt`
1. [Step 3:](#list_constants) Specify constants and variables in Hamiltonian expression
1. [Step 4:](#list_free_symbols) Extract free symbols
1. [Step 5:](#convert_to_func) Convert variables to function notation; e.g., `var` goes to `var(xx)`
1. [Step 6:](#differentiate) Differentiate with respect to `xx`
1. [Step 7:](#remove_zeros) Remove derivatives (of constants) that evaluate to zero, simplifying derivative expressions
1. [Step 8:](#store_results) Store partial derivatives to SymPy notebook `partial_derivatives.txt-VALIDATION.txt`
1. [Step 9:](#numpy_function) Create numpy function to output exact derivatives for all 12 dynamic variables 
1. [Step 10:](#code_validation) Validate against LALSuite and trusted `SEOBNR_Derivative_Routine` NRPy+ module
1. [Step 11:](#spherical_orbit) Spherical Orbit Test
1. [Step 11:](#latex_pdf_output) Output this notebook to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Initialize core Python/NRPy+ modules \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Let's start by importing all the needed modules from Python/NRPy+ and creating the output directory (if it does not already exist). Note that since the expression 'Q' appears in the Hamiltonian and is a protected symbol in sympy we must create a new global dictionary to ensure that sympy does not spit out errors.

In [1]:
# Step 1.a: import all needed modules from Python/NRPy+:
import sympy as sp                # SymPy: The Python computer algebra package upon which NRPy+ depends
import sys, os     # Standard Python modules for multiplatform OS-level functions
sys.path.append('../')
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

from outputC import superfast_uniq, lhrh, outputC      # Remove duplicate entries from a Python array; store left- and right-
                                              #   hand sides of mathematical expressions
                                              #   create optimized C expressions

# As of April 2021, "sp.sympify("Q+1")" fails because Q is a reserved keyword.
#   This is the workaround, courtesy Ken Sible.
custom_global_dict = {}
exec('from sympy import *', custom_global_dict)
del custom_global_dict['Q']

# Step 1.b: Check for a sufficiently new version of SymPy (for validation)
# Ignore the rc's and b's for release candidates & betas.
sympy_version = sp.__version__.replace('rc', '...').replace('b', '...')
sympy_version_decimal = float(int(sympy_version.split(".")[0]) + int(sympy_version.split(".")[1])/10.0)
print(sympy_version_decimal)
if sympy_version_decimal > 1.2:
    custom_parse_expr = lambda expr: sp.parse_expr(expr, global_dict=custom_global_dict)
else:
    custom_parse_expr = lambda expr: sp.sympify(expr)

if sympy_version_decimal < 1.2:
    print('Error: NRPy+ does not support SymPy < 1.2')
    sys.exit(1)

# Step 1.c: Name of the directory containing the input/output file
inputdir = "Hamiltonian"
outputdir = "Derivatives"

cmd.mkdir(outputdir)

1.8


<a id='read_expressions'></a>

# Step 2: Read in Hamiltonian expressions from `Sympy_Hreal_on_Bottom.txt` \[Back to [top](#toc)\]
$$\label{read_expressions}$$

We read in the expressions of which we will compute partial derivatives in a single large string before splitting the string by line (carriage return) and by "=".  Doing so allows us to manipulate the right- and left-hand sides of the expressions appropriately.  We store the left- and right-hand sides in the array `lr`, which consists of `lhrh` arrays with left-hand sides `lhs` and right-hand sides `rhs`.  Note that `Lambda` is a protected keyword in Python, so the variable $\Lambda$ in the Hamiltonian is renamed `Lamb`.

In [2]:
# Step 2.a: Read in expressions as a (single) string
with open(os.path.join(inputdir,'v5HM_Hamiltonian-Hreal_on_Bottom.txt'), 'r') as file:
    expressions_as_lines = file.readlines()

# Step 2.b: Create and populate the "lr" array, which separates each line into left- and right-hand sides
#   Each entry is a string of the form lhrh(lhs='',rhs='')
lr = []

for i in range(len(expressions_as_lines)):
    # Ignore lines with 2 or fewer characters and those starting with #
    if len(expressions_as_lines[i]) > 2 and expressions_as_lines[i][0] != "#":
        # Split each line by its equals sign
        split_line = expressions_as_lines[i].split("=")
        # Append the line to "lr", removing spaces, "sp." prefixes, and replacing Lambda->Lamb
        #   (Lambda is a protected keyword):
        lr.append(lhrh(lhs=split_line[0].replace(" ","").replace("Lambda","Lamb"),
                       rhs=split_line[1].replace(" ","").replace("sp.","").replace("Lambda","Lamb")))

# Step 2.c: Separate and sympify right- and left-hand sides into separate arrays
lhss = []
rhss = []
for i in range(len(lr)):
    lhss.append(custom_parse_expr(lr[i].lhs))
    rhss.append(custom_parse_expr(lr[i].rhs))

<a id='list_constants'></a>

# Step 3: Specify constants and variables in Hamiltonian expression \[Back to [top](#toc)\]
$$\label{list_constants}$$

We read in and declare as SymPy symbols the constant values; derivatives with respect to these variables will be set to zero.  We then read in the variables with respect to which we want to take derivatives and declare those as SymPy variables as well.

In [3]:
# Step 3.a: Create `input_constants` array and populate with SymPy symbols
m1,m2,chi1,chi2 = sp.symbols('m1 m2 chi1 chi2',real=True)

input_constants = [m1,m2,chi1,chi2]
# Step 3.b: Create `dynamic_variables` array and populate with SymPy symbols
r,prstar,pphi = sp.symbols("r prstar pphi", real=True)
dynamic_variables = [r,prstar,pphi]

<a id='list_free_symbols'></a>

# Step 4: Extract free symbols \[Back to [top](#toc)\]
$$\label{list_free_symbols}$$

By ''free symbols'' we mean the variables in the right-hand sides.  We first create a list of all such terms (using SymPy's built-in free_symbol attribute), including duplicates, and then strip the duplicates.  We then remove input constants from the symbol list.

In [4]:
# Step 4.a: Prepare array of "free symbols" in the right-hand side expressions
full_symbol_list_with_dups = []
for i in range(len(lr)):
    for variable in rhss[i].free_symbols:
        full_symbol_list_with_dups.append(variable)

# Step 4.b: Remove duplicate free symbols
full_symbol_list = superfast_uniq(full_symbol_list_with_dups)

# Step 4.c: Remove input constants from symbol list
for inputconst in input_constants:
    for symbol in full_symbol_list:
        if str(symbol) == str(inputconst):
            full_symbol_list.remove(symbol)

<a id='convert_to_func'></a>

# Step 5: Convert variables to function notation; e.g., `var` goes to `var(xx)` \[Back to [top](#toc)\]
$$\label{convert_to_func}$$

In order to compute the partial derivative of each right-hand side, we mark each variable (left-hand side) and each free symbol (in right-hand sides) as a function with the dynamic variables as arguments.

In [5]:
# Step 5.a: Convert each left-hand side to function notation
#   while separating and simplifying left- and right-hand sides
xx = sp.Symbol('xx',real=True)
func = []
for i in range(len(lr)):
    func.append(sp.sympify(sp.Function(lr[i].lhs,real=True)(xx)))

# Step 5.b: Mark each free variable as a function with arguments as the dynamic variables
full_function_list = []
for symb in full_symbol_list:
    func = sp.sympify(sp.Function(str(symb),real=True)(xx))
    full_function_list.append(func)
    for i in range(len(rhss)):
        for var in rhss[i].free_symbols:
            if str(var) == str(symb):
                rhss[i] = rhss[i].subs(var,func)
print(full_function_list)

[r(xx), M(xx), eta(xx), pphi(xx), prstar(xx), u(xx), Qnosterm2(xx), Qnosterm3(xx), Qnosterm1(xx), a6(xx), ap(xx), Anons(xx), Dnons(xx), xi(xx), delta(xx), pr(xx), am(xx), QalignSS(xx), Qnos(xx), BnpalignSS(xx), AalignSS(xx), dSO(xx), Qalign(xx), Aalign(xx), Balignnp(xx), Bkerreqnp(xx), Galigna3(xx), gap(xx), SOcalib(xx), gam(xx), Hodd(xx), Heven(xx), Heff(xx)]


<a id='differentiate'></a>

# Step 6: Differentiate with respect to dynamic variables \[Back to [top](#toc)\]
$$\label{differentiate}$$

Now we differentiate the right-hand expressions with respect to the dynamic variabls.  We use the SymPy $\texttt{diff}$ command, differentiating with respect to each dynamic variable.  After so doing, we remove $\texttt{(xx)}$, where 'xx' represents a dynamic variable and "Derivative" (which is output by $\texttt{diff}$), and use "prm_xx" suffix to denote the derivative with respect to $\texttt{xx}$.

In [6]:
# Step 6: Use SymPy's diff function to differentiate right-hand sides with respect to dynamic variables
#   and append "prm" notation to left-hand sides
lhss_deriv = []
rhss_deriv = []
for i in range(len(rhss)):
    lhss_deriv.append(custom_parse_expr(str(lhss[i])+"prm1"))
    newrhs = custom_parse_expr(str(sp.diff(rhss[i],xx)).replace("(xx)","").replace(", xx","prm1").replace("Derivative",""))
    rhss_deriv.append(newrhs)    

<a id='remove_zeros'></a>

# Step 7: Remove derivatives (of constants) that evaluate to zero, simplifying derivative expressions \[Back to [top](#toc)\]
$$\label{remove_zeros}$$

We declare a function to simply the derivative expressions.  In particular, we want to remove terms equal to zero.

In [7]:
# Step 7.a: Define derivative simplification function
def simplify_deriv(lhss_deriv,rhss_deriv):
    # Copy expressions into another array
    lhss_deriv_simp = []
    rhss_deriv_simp = []
    for i in range(len(rhss_deriv)):
        lhss_deriv_simp.append(lhss_deriv[i])
        rhss_deriv_simp.append(rhss_deriv[i])
    # If a right-hand side is 0, substitute value 0 for the corresponding left-hand side in later terms
    for i in range(len(rhss_deriv_simp)):
        if rhss_deriv_simp[i] == 0:
            for j in range(i+1,len(rhss_deriv_simp)):
                for var in rhss_deriv_simp[j].free_symbols:
                    if str(var) == str(lhss_deriv_simp[i]):
                        rhss_deriv_simp[j] = rhss_deriv_simp[j].subs(var,0)
    zero_elements_to_remove = []
    # Create array of indices for expressions that are zero
    for i in range(len(rhss_deriv_simp)):
        if rhss_deriv_simp[i] == sp.sympify(0):
            zero_elements_to_remove.append(i)
    # When removing terms that are zero, we need to take into account their new index (after each removal)
    count = 0
    for i in range(len(zero_elements_to_remove)):
        del lhss_deriv_simp[zero_elements_to_remove[i]+count]
        del rhss_deriv_simp[zero_elements_to_remove[i]+count]
        count -= 1
    return lhss_deriv_simp,rhss_deriv_simp

# Step 7.b: Call the simplication function and then copy results
lhss_deriv_simp,rhss_deriv_simp = simplify_deriv(lhss_deriv,rhss_deriv)
lhss_deriv = lhss_deriv_simp
rhss_deriv = rhss_deriv_simp

<a id='list_free_symbols2'></a>

# Step 4: Re-extract free symbols \[Back to [top](#toc)\]
$$\label{list_free_symbols2}$$

We do the same as [Step 4](#list_free_symbols) with the newly generated first derivative terms in order to construct second derivatives, which are needed for the initial conditions.

In [8]:
# Step 4.a: Prepare array of "free symbols" in the right-hand side expressions
rhss_deriv1 = []
lhss_deriv1 = []

for i in range(len(rhss_deriv)):
    rhss_deriv1.append(rhss_deriv[i])
    lhss_deriv1.append(lhss_deriv[i])

full_symbol_list_with_dups = full_symbol_list
for i in range(len(lhss_deriv1)):
    for variable in rhss_deriv1[i].free_symbols:
        full_symbol_list_with_dups.append(variable)

# Step 4.b: Remove duplicate free symbols
full_symbol_list_2 = superfast_uniq(full_symbol_list_with_dups)

# Step 4.c: Remove input constants from symbol list
for inputconst in input_constants:
    for symbol in full_symbol_list_2:
        if str(symbol) == str(inputconst):
            full_symbol_list_2.remove(symbol)

<a id='convert_to_func'></a>

# Step 5: Convert variables to function notation; e.g., `var` goes to `var(xx)` \[Back to [top](#toc)\]
$$\label{convert_to_func}$$

In order to compute the partial derivative of each right-hand side, we mark each variable (left-hand side) and each free symbol (in right-hand sides) as a function with the dynamic variables as arguments.

In [9]:
# Step 5.a: Convert each left-hand side to function notation
#   while separating and simplifying left- and right-hand sides
yy = sp.Symbol('yy',real=True)
func = []
#for i in range(len(lhss_deriv)):
#    func.append(sp.sympify(sp.Function(lhss_deriv[i],real=True)(xx)))

# Step 5.b: Mark each free variable as a function with arguments as the dynamic variables
full_function_list = []
for symb in full_symbol_list_2:
    func = sp.sympify(sp.Function(str(symb),real=True)(yy))
    full_function_list.append(func)
    for i in range(len(rhss_deriv1)):
        for var in rhss_deriv1[i].free_symbols:
            if str(var) == str(symb):
                rhss_deriv1[i] = rhss_deriv1[i].subs(var,func)

print(full_function_list)

[r(yy), M(yy), eta(yy), pphi(yy), prstar(yy), u(yy), Qnosterm2(yy), Qnosterm3(yy), Qnosterm1(yy), a6(yy), ap(yy), Anons(yy), Dnons(yy), xi(yy), delta(yy), pr(yy), am(yy), QalignSS(yy), Qnos(yy), BnpalignSS(yy), AalignSS(yy), dSO(yy), Qalign(yy), Aalign(yy), Balignnp(yy), Bkerreqnp(yy), Galigna3(yy), gap(yy), SOcalib(yy), gam(yy), Hodd(yy), Heven(yy), Heff(yy), rprm1(yy), pphiprm1(yy), uprm1(yy), prstarprm1(yy), Qnosterm2prm1(yy), Qnosterm1prm1(yy), Qnosterm3prm1(yy), Dnonsprm1(yy), Anonsprm1(yy), xiprm1(yy), prprm1(yy), QalignSSprm1(yy), Qnosprm1(yy), BnpalignSSprm1(yy), AalignSSprm1(yy), Aalignprm1(yy), Qalignprm1(yy), Bkerreqnpprm1(yy), Balignnpprm1(yy), gamprm1(yy), SOcalibprm1(yy), Galigna3prm1(yy), gapprm1(yy), Hoddprm1(yy), Hevenprm1(yy), Heffprm1(yy)]


<a id='differentiate'></a>

# Step 6: Differentiate with respect to dynamic variables \[Back to [top](#toc)\]
$$\label{differentiate}$$

Now we differentiate the right-hand expressions with respect to the dynamic variabls.  We use the SymPy $\texttt{diff}$ command, differentiating with respect to each dynamic variable.  After so doing, we remove $\texttt{(xx)}$, where 'xx' represents a dynamic variable and "Derivative" (which is output by $\texttt{diff}$), and use "prm_xx" suffix to denote the derivative with respect to $\texttt{xx}$.

In [10]:
# Step 6: Use SymPy's diff function to differentiate right-hand sides with respect to dynamic variables
#   and append "prm" notation to left-hand sides
lhss_deriv2 = []
rhss_deriv2 = []
for i in range(len(rhss_deriv1)):
    lhss_deriv2.append(custom_parse_expr(str(lhss_deriv1[i])+"prm2"))
    newrhs = custom_parse_expr(str(sp.diff(rhss_deriv1[i],yy)).replace("(yy)","").replace(", yy","prm2").replace("Derivative",""))
    rhss_deriv2.append(newrhs)    

<a id='remove_zeros'></a>

# Step 7: Remove derivatives (of constants) that evaluate to zero, simplifying derivative expressions \[Back to [top](#toc)\]
$$\label{remove_zeros}$$

We declare a function to simply the derivative expressions.  In particular, we want to remove terms equal to zero.

In [11]:
# Step 7.b: Call the simplication function and then copy results
print(lhss_deriv2,rhss_deriv2)
lhss_deriv2_simp,rhss_deriv2_simp = simplify_deriv(lhss_deriv2,rhss_deriv2)
lhss_deriv2 = lhss_deriv2_simp
rhss_deriv2 = rhss_deriv2_simp

[uprm1prm2, gamprm1prm2, gapprm1prm2, Qnosterm3prm1prm2, Qnosterm2prm1prm2, Qnosterm1prm1prm2, Qnosprm1prm2, Dnonsprm1prm2, Anonsprm1prm2, xiprm1prm2, prprm1prm2, QalignSSprm1prm2, BnpalignSSprm1prm2, AalignSSprm1prm2, Qalignprm1prm2, Balignnpprm1prm2, Bkerreqnpprm1prm2, Aalignprm1prm2, Galigna3prm1prm2, SOcalibprm1prm2, Hevenprm1prm2, Hoddprm1prm2, Heffprm1prm2, Hrealprm1prm2] [-rprm1prm2/r**2 + 2*rprm1*rprm2/r**3, 9*etaprm2*pphi**2*rprm1/(16*r**3) - 9*etaprm2*pphi*pphiprm1/(16*r**2) - 11*etaprm2*rprm1/(32*r**2) - pphi**4*rprm1*(75*eta*etaprm2/32 - 45*etaprm2/32)/r**5 - pphi**4*rprm1prm2*(75*eta**2/64 - 45*eta/32 - 105/64)/r**5 + pphi**4*rprm1*rprm2*(375*eta**2/64 - 225*eta/32 - 525/64)/r**6 + pphi**3*pphiprm1*(75*eta*etaprm2/32 - 45*etaprm2/32)/r**4 - pphi**3*pphiprm1*rprm2*(75*eta**2/16 - 45*eta/8 - 105/16)/r**5 + pphi**3*pphiprm1prm2*(75*eta**2/64 - 45*eta/32 - 105/64)/r**4 - pphi**3*pphiprm2*rprm1*(75*eta**2/16 - 45*eta/8 - 105/16)/r**5 + pphi**2*pphiprm1*pphiprm2*(225*eta**2/64 -

<a id='partial_derivative'></a>

# Step 8: Simplify derivatives with respect to specific variables \[Back to [top](#toc)\]
$$\label{partial_derivative}$$

In [Step 6](#differentiate) we took a generic derivative of each expression, assuming all variables were functions of `xx`.  We now define a function that will select a specific dynamic variable (element of `dynamic_variables`) and set the derivative of the variable to 1 and all others to 0.

In [12]:
# Step 8.a: Define onevar derivative function
# deriv_onevar() replaces variable derivatives with 1 or 0 depending on which partial derivaitve is computed.  For
# example, pass 'xprm=1' to replace each instance of 'xprm' with 1 and 'qprm' with 0 for each q in (y,z,p1,p2,p3,S1x,
# S1y,S1z,S2x,S2y,S2z).  This produces expressions which compute the partial derivative of the Hamiltonian with respect
# to x.
def deriv_onevar(lhss_deriv, rhss_deriv, rprm=0, prstarprm=0, pphiprm=0):
    if rprm + prstarprm + pphiprm != 1:
        print("deriv_onevar() cannot take more than one derivative at a time!")
        sys.exit()

    # Create 'new' arrays to store and manipulate derivative terms.
    lhss_deriv_new = []
    rhss_deriv_new = []
    # Append derivative terms to 'new' arrays
    for i in range(len(rhss_deriv)):
        lhss_deriv_new.append(lhss_deriv[i])
        rhss_deriv_new.append(rhss_deriv[i])
    # Replace each instance of 'qprm', q in (x,y,z,p1,p2,p3,S1x,S1y,S1z,S2x,S2y,S2z), with either 0 or 1.
    for i in range(len(rhss_deriv_new)):
        for var in rhss_deriv_new[i].free_symbols:
            if str(var) == "rprm1":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var, rprm)
            elif str(var) == "prstarprm1":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var, prstarprm)
            elif str(var) == "pphiprm1":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var, pphiprm)
    # Simplify the derivative expressions with simplify_deriv().
    lhss_deriv_simp, rhss_deriv_simp = simplify_deriv(lhss_deriv_new, rhss_deriv_new)
    # Return simplified derivative expression.
    return lhss_deriv_simp, rhss_deriv_simp

# Step 8.b: Define twovar derivative function
# deriv_onevar() replaces variable derivatives with 1 or 0 depending on which partial derivaitve is computed.  For
# example, pass 'xprm=1' to replace each instance of 'xprm' with 1 and 'qprm' with 0 for each q in (y,z,p1,p2,p3,S1x,
# S1y,S1z,S2x,S2y,S2z).  This produces expressions which compute the partial derivative of the Hamiltonian with respect
# to x.
def deriv_twovar(lhss_deriv, rhss_deriv, rprm1=0, prstarprm1=0, pphiprm1=0, rprm2=0, prstarprm2=0, pphiprm2=0):
    if rprm1 + prstarprm1 + pphiprm1 != 1:
        print("deriv_twovar() cannot take more than two derivatives at a time!")
        sys.exit()
    if rprm2 + prstarprm2 + pphiprm2 != 1:
        print("deriv_twovar() cannot take more than two derivatives at a time!")
        sys.exit()

    # Create 'new' arrays to store and manipulate derivative terms.
    lhss_deriv_new = []
    rhss_deriv_new = []
    # Append derivative terms to 'new' arrays
    for i in range(len(rhss_deriv)):
        lhss_deriv_new.append(lhss_deriv[i])
        rhss_deriv_new.append(rhss_deriv[i])
    for i in range(len(rhss_deriv_new)):
        # Replace each instance of 'qprm1', q in (x,y,z,p1,p2,p3,S1x,S1y,S1z,S2x,S2y,S2z), with either 0 or 1.
        for var in rhss_deriv_new[i].free_symbols:
            if str(var) == "rprm1":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var, rprm1)
            elif str(var) == "prstarprm1":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var, prstarprm1)
            elif str(var) == "pphiprm1":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var, pphiprm1)
        # Replace each instance of 'qprm2', q in (x,y,z,p1,p2,p3,S1x,S1y,S1z,S2x,S2y,S2z), with either 0 or 1.
            if str(var) == "rprm2":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var, rprm2)
            elif str(var) == "prstarprm2":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var, prstarprm2)
            elif str(var) == "pphiprm2":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var, pphiprm2)
        # Replace each instance of 'qprm1prm2' with zero
            if str(var) == "rprm1prm2":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var, 0)
            elif str(var) == "prstarprm1prm2":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var, 0)
            elif str(var) == "pphiprm1prm2":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var, 0)
    # Simplify the derivative expressions with simplify_deriv().
    lhss_deriv_simp, rhss_deriv_simp = simplify_deriv(lhss_deriv_new, rhss_deriv_new)
    # Return simplified derivative expression.
    return lhss_deriv_simp, rhss_deriv_simp

<a id='store_results'></a>

# Step 9: Store partial derivatives to SymPy notebook `partial_derivatives.txt-VALIDATION.txt` \[Back to [top](#toc)\]
$$\label{store_results}$$

We write the resulting derivatives in SymPy syntax.  Each partial derivative is output in its own file, in a similar format to the input expressions.

In [13]:
# Step 9: Output original expression and each partial derivative expression in SymPy snytax
with open(os.path.join(outputdir,'v5HM_first_partial_derivatives.txt'), 'w') as output:
    for i in range(len(lr)):
        right_side = lr[i].rhs
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("EulerGamma","sp.EulerGamma").replace("Rational(","sp.Rational(")
        #print(str(lr[i].lhs)+" = "+right_side_in_sp) 
        output.write(str(lr[i].lhs)+" = "+right_side_in_sp)
    output.write("\n")
    for i in range(len(lhss_deriv)):
        right_side = str(rhss_deriv[i])
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("EulerGamma","sp.EulerGamma").replace("Rational(","sp.Rational(").replace("prm1","_prm1")
        output.write(str(lhss_deriv[i]).replace("prm1","_prm1")+" = "+right_side_in_sp+"\n")
            
# Step 10: Output original expression and each partial derivative expression in SymPy snytax
with open(os.path.join(outputdir,'v5HM_second_partial_derivatives.txt'), 'w') as output:
    for i in range(len(lr)):
        right_side = lr[i].rhs
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("EulerGamma","sp.EulerGamma").replace("Rational(","sp.Rational(")
        #print(str(lr[i].lhs)+" = "+right_side_in_sp) 
        output.write(str(lr[i].lhs)+" = "+right_side_in_sp)
    output.write("\n")
    for i in range(len(lhss_deriv)):
        right_side = str(rhss_deriv[i])
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("EulerGamma","sp.EulerGamma").replace("Rational(","sp.Rational(").replace("prm1","_prm1")
        output.write(str(lhss_deriv[i]).replace("prm1","_prm1")+" = "+right_side_in_sp+"\n")
    output.write("\n")    
    for i in range(len(lhss_deriv)):
        right_side = str(rhss_deriv[i])
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("EulerGamma","sp.EulerGamma").replace("Rational(","sp.Rational(").replace("prm1","_prm2")
        output.write(str(lhss_deriv[i]).replace("prm1","_prm2")+" = "+right_side_in_sp+"\n")
    output.write("\n")
    for i in range(len(lhss_deriv2)):
        right_side = str(rhss_deriv2[i])
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("EulerGamma","sp.EulerGamma").replace("Rational(","sp.Rational(").replace("prm1","_prm1").replace("prm2","_prm2")
        output.write(str(lhss_deriv2[i]).replace("prm1","_prm1").replace("prm2","_prm2")+" = "+right_side_in_sp+"\n")

<a id='specific_derivatives'></a>

# Step 10: Compute specific derivatives  \[Back to [top](#toc)\]
$$\label{specific _derivatives}$$

We write resulting specific derivatives in sympy syntax. The derivatives needed for initial conditions, integrating the trajectory, and computing the waveform are as follows

- For the conservative equations of motion, $\frac{\partial \mathcal{H}}{\partial r}$, $\frac{\partial \mathcal{H}}{\partial p_{r}}$, $\frac{\partial \mathcal{H}}{\partial p_{\phi}}$.
- For the waveform and radiation reaction, $\Omega_{\rm circ} \equiv \left.\frac{\partial \mathcal{H}}{\partial p_{\phi}}\right|_{p_{r} = 0}$.
- For the initial conditions, $\frac{\partial^2 \mathcal{H}}{\partial r^2}$, $\frac{\partial^2 \mathcal{H}}{\partial r \partial p_{\phi}}$

In [16]:
# Step 10: output exact derivatives for all required cases
lhss_deriv_r, rhss_deriv_r = deriv_onevar(lhss_deriv, rhss_deriv, rprm = 1, prstarprm = 0, pphiprm = 0)
lhss_deriv_prstar, rhss_deriv_prstar = deriv_onevar(lhss_deriv, rhss_deriv, rprm = 0, prstarprm = 1, pphiprm = 0)
lhss_deriv_pphi, rhss_deriv_pphi = deriv_onevar(lhss_deriv, rhss_deriv, rprm = 0, prstarprm = 0, pphiprm = 1)
lhss_deriv_r_r, rhss_deriv_r_r = deriv_twovar(lhss_deriv2, rhss_deriv2, rprm1 = 1, prstarprm1 = 0, pphiprm1 = 0,
                                              rprm2 = 1, prstarprm2 = 0, pphiprm2 = 0)
lhss_deriv_r_pphi, rhss_deriv_r_pphi = deriv_twovar(lhss_deriv2, rhss_deriv2, rprm1 = 1, prstarprm1 = 0, pphiprm1 = 0,
                                              rprm2 = 0, prstarprm2 = 0, pphiprm2 = 1)

# output the omega function
omega_rhs = []
omega_lhs = []

for i in range(len(lhss_deriv_pphi)):
    omega_lhs.append(lhss_deriv_pphi[i])
    omega_rhs.append(rhss_deriv_pphi[i])
    for var in omega_rhs[i].free_symbols:
        if str(var) == "prstar":
            omega_rhs[i] = omega_rhs[i].subs(var,0)

omega_lhs_simplified , omega_rhs_simplified = simplify_deriv(omega_lhs,omega_rhs)
omega_lhs = omega_lhs_simplified
omega_rhs = omega_rhs_simplified

# Store Variables

with open(os.path.join(outputdir,'v5HM_Hamiltonian_Derivative_dr.txt'), 'w') as output:
    for i in range(len(lr)):
        right_side = lr[i].rhs
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("EulerGamma","sp.EulerGamma").replace("Rational(","sp.Rational(")
        output.write(str(lr[i].lhs)+" = "+right_side_in_sp)
    output.write("\n")
    for i in range(len(lhss_deriv_r)):
        right_side = str(rhss_deriv_r[i])
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("EulerGamma","sp.EulerGamma").replace("Rational(","sp.Rational(").replace("prm1","_prmr")
        output.write(str(lhss_deriv_r[i]).replace("prm1","_prmr")+" = "+right_side_in_sp+"\n")

with open(os.path.join(outputdir,'v5HM_Hamiltonian_Derivative_dprstar.txt'), 'w') as output:
    for i in range(len(lr)):
        right_side = lr[i].rhs
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("EulerGamma","sp.EulerGamma").replace("Rational(","sp.Rational(")
        output.write(str(lr[i].lhs)+" = "+right_side_in_sp)
    output.write("\n")
    for i in range(len(lhss_deriv_prstar)):
        right_side = str(rhss_deriv_prstar[i])
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("EulerGamma","sp.EulerGamma").replace("Rational(","sp.Rational(").replace("prm1","_prmprstar")
        output.write(str(lhss_deriv_prstar[i]).replace("prm1","_prmprstar")+" = "+right_side_in_sp+"\n")

with open(os.path.join(outputdir,'v5HM_Hamiltonian_Derivative_dpphi.txt'), 'w') as output:
    for i in range(len(lr)):
        right_side = lr[i].rhs
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("EulerGamma","sp.EulerGamma").replace("Rational(","sp.Rational(")
        output.write(str(lr[i].lhs)+" = "+right_side_in_sp)
    output.write("\n")
    for i in range(len(lhss_deriv_pphi)):
        right_side = str(rhss_deriv_pphi[i])
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("EulerGamma","sp.EulerGamma").replace("Rational(","sp.Rational(").replace("prm1","_prmpphi")
        output.write(str(lhss_deriv[i]).replace("prm1","_prmpphi")+" = "+right_side_in_sp+"\n")

with open(os.path.join(outputdir,'v5HM_Hamiltonian_Omega.txt'), 'w') as output:
    for i in range(len(lr)):
        right_side = lr[i].rhs
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("EulerGamma","sp.EulerGamma").replace("Rational(","sp.Rational(")
        output.write(str(lr[i].lhs)+" = "+right_side_in_sp)
    output.write("\n")
    for i in range(len(omega_lhs)):
        right_side = str(omega_rhs[i])
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("EulerGamma","sp.EulerGamma").replace("Rational(","sp.Rational(").replace("prm1","_prmpphi_preq0")
        output.write(str(omega_lhs[i]).replace("prm1","_prmpphi_preq0")+" = "+right_side_in_sp+"\n")
    
with open(os.path.join(outputdir,'v5HM_Hamiltonian_Derivative_dr.txt'), 'w') as output:
    for i in range(len(lr)):
        right_side = lr[i].rhs
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("EulerGamma","sp.EulerGamma").replace("Rational(","sp.Rational(")
        output.write(str(lr[i].lhs)+" = "+right_side_in_sp)
    output.write("\n")
    for i in range(len(lhss_deriv_r)):
        right_side = str(rhss_deriv_r[i])
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("EulerGamma","sp.EulerGamma").replace("Rational(","sp.Rational(").replace("prm1","_prmr")
        output.write(str(lhss_deriv[i]).replace("prm1","_prmr")+" = "+right_side_in_sp+"\n")
    for i in range(len(lhss_deriv_pphi)):
        right_side = str(rhss_deriv_pphi[i])
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("EulerGamma","sp.EulerGamma").replace("Rational(","sp.Rational(").replace("prm1","_prmpphi")
        output.write(str(lhss_deriv[i]).replace("prm1","_prmpphi")+" = "+right_side_in_sp+"\n")
    
